# Communication Channel

This example demonstrates how to create
a connection from one neuronal ensemble to another
that behaves like a communication channel
(that is, it transmits information without changing it).

Network diagram:

      [Input] ---> (FPGA input) ---> (FPGA output)

An abstract input signal is fed into
the neuronal ensemble which is built remotely on the FPGA.
The ensemble on the FPGA passes takes the input and passes
this value across the neurons to the output.
The result is that activity in the output ensemble
encodes the value from the input.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

import nengo

import nengo_fpga
from nengo_fpga.networks import FpgaPesEnsembleNetwork

## Step 1: Choose a Device

Select the FPGA device on which you wish to run the remote
`FpgaPesEnsembleNetwork`. This name corresponds with the name
in your `fpga_config` file. Recall the device name
is the name in square brackets which precedes the
settings, for example, **[de1]** or **[pynq]**.

In [ ]:
board = 'de1'  # Change this to your desired device name

## Step 2: Create the Network

We will build a network that receives a 2-dimensional input.
First let's define the input function we want to pass to the network.

In [ ]:
def input_func(t):
    return [np.sin(t * 2*np.pi), np.cos(t * 2*np.pi)]

Now we can create the rest of the network.

In [ ]:
# Create a 'model' object to which we can add
# ensembles, connections, etc.
model = nengo.Network(label="Communication Channel")
with model:
    # Arbitrary reference signal defined above
    input_node = nengo.Node(input_func, label='input signal')

    # FPGA neural ensemble
    fpga_ens = FpgaPesEnsembleNetwork(
        board, n_neurons=50, dimensions=2, learning_rate=0,
        label="FPGA ensemble")
    
    # Uncomment the following line to use spiking neuron
    # fpga_ens.ensemble.neuron_type = nengo.SpikingRectifiedLinear()
    
    # Connect the input to the FPGA ensemble
    # (the FPGA input is automatically passed to the output,
    # no need for an explicit connection)
    nengo.Connection(input_node, fpga_ens.input)

## Step 3: Add Probes to Collect Data

Even this simple model involves many quantities
that change over time, such as membrane potentials of individual neurons.
Typically there are so many variables in a simulation
that it is not practical to store them all.
If we want to plot or analyze data from the simulation
we have to "probe" the signals of interest.

Many of the internal dynamics of the FPGA ensemble are not 
probeable since collecting and transmitting all of these values
would slow down the simulation considerably. However,
we still have access to the input and output activity which
is enough to illustrate the network functionality.

In [ ]:
with model:
    input_p = nengo.Probe(input_node, synapse=0.01)
    output_p = nengo.Probe(fpga_ens.output, synapse=0.01)

## Step 4: Run the Model!

Here we simply switch out the standard `nengo.Simulator`
for the `nengo_fpga.simulator`

In [ ]:
with nengo_fpga.Simulator(model) as sim:
    sim.run(2)  # Run for 2 seconds

## Step 5: Plot the Results

In [ ]:
plt.figure(figsize=(16, 4))
plt.subplot(1, 2, 1)
plt.title("Input")
plt.plot(sim.trange(), sim.data[input_p])
plt.ylim(-1.1, 1.1)
plt.legend(("Input[0]", "Input[1]"), loc='best')
plt.subplot(1, 2, 2)
plt.title("FPGA Output")
plt.plot(sim.trange(), sim.data[output_p])
plt.ylim(-1.1, 1.1)
plt.legend(("Input[0]", "Input[1]"), loc='best');

These plots show the reference input signal on the left and
the reconstruction of this signal decoded from the neuron activity
of the FPGA ensemble on the right.

## Step 6: Experiment with this Model

### Input Function

Try changing the input function above. As long as the function
outputs a 2-dimensional value the model should work but
you may see the reconstructed signal deteriorate as the
range and frequency of your input function increase.

### Neuron Type

The default neuron type is Rectified Linear Units (ReLU).
NengoFPGA also supports Spiking Rectified Linear units,
also known as Integrate and Fire (IF) neurons.
Try uncommenting the line noted in the model definition
in Step 2 and rerunning the simulation.